# Objective: Predict World Happiness Rankings 

What makes the citizens of one country more happy than the citizens of other countries?  Do variables measuring perceptions of corruption, GDP, maintaining a healthy lifestyle, or social support associate with a country's happiness ranking?  

Let's use the United Nation's World Happiness Rankings country level data to experiment with models that predict happiness rankings well.


---

**Data**: 2019 World Happiness Survey Rankings
*(Data can be found on Advanced Projects in ML courseworks site)*

**Features**
*   Country or region
*   GDP per capita
*   Social support
*   Healthy life expectancy
*   Freedom to make life choices
*   Generosity
*   Perceptions of corruption

**Target**
*   Happiness_level (Very High = Top 20% and Very Low = Bottom 20%)

Source: https://worldhappiness.report/




# Mini-Hackathon In Class Tasks



1.   Build, save, and submit at least one Keras model.
2.   Build, save, and submit at least one Scikit-learn model.
3.   Seek advice through collaboration via Github:

*      Save notebook w/ best model to private repo
*      Invite a collaborator
*      Collaborator should submit at least two issues w/ suggestions for model improvement

4.   If time, improve model further!











# Import the data




In [1]:
! pip install scikit-learn --upgrade # load newest version of sklearn

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.22.1)


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

data=pd.read_csv("worldhappiness2019_onenewvariable.csv")

data.head()

,Happiness_level,Country or region,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Terrorist_attacks
0,Average,Hong Kong,1.438,1.277,1.122,0.440,0.258,0.287,585.434783
1,Average,Moldova,0.685,1.328,0.739,0.245,0.181,0.000,2.000000
2,Average,North Macedonia,0.983,1.294,0.838,0.345,0.185,0.034,585.434783
3,Average,Northern Cyprus,1.263,1.252,1.042,0.417,0.191,0.162,585.434783
4,Average,Russia,1.183,1.452,0.726,0.334,0.082,0.031,559.000000


In [3]:
# Truncated and cleaned up region data to merge (Week 4 folder)
regiondata=pd.read_csv("region.csv")

regiondata.head()

,name,region,sub-region
0,Afghanistan,Asia,Southern Asia
1,Åland Islands,Europe,Northern Europe
2,Albania,Europe,Southern Europe
3,Algeria,Africa,Northern Africa
4,American Samoa,Oceania,Polynesia


In [4]:
# Clean up final data

mergedata=pd.merge(data, regiondata, how='left', left_on='Country or region', right_on='name')
# Check for missing values (there won't be any given that I have already cleaned up the region data)
mergedata.loc[pd.isnull(mergedata).iloc[:,9]].to_csv("missing.csv",index=False)

# clean up final region data
X=mergedata.drop(['Happiness_level'],axis=1)
X=X.drop(['name'],axis=1)
X=X.drop(['Country or region'],axis=1)
X=X.drop(['sub-region'],axis=1)

X


,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Terrorist_attacks,region
0,1.438,1.277,1.122,0.440,0.258,0.287,585.434783,Asia
1,0.685,1.328,0.739,0.245,0.181,0.000,2.000000,Europe
2,0.983,1.294,0.838,0.345,0.185,0.034,585.434783,Europe
3,1.263,1.252,1.042,0.417,0.191,0.162,585.434783,Asia
4,1.183,1.452,0.726,0.334,0.082,0.031,559.000000,Europe
...,...,...,...,...,...,...,...,...
151,0.274,0.757,0.505,0.142,0.275,0.078,1419.095238,Africa
152,1.120,1.402,0.798,0.498,0.215,0.060,125.611111,Africa
153,1.362,1.368,0.871,0.536,0.255,0.110,95.000000,Asia
154,1.572,1.463,1.141,0.556,0.271,0.453,125.611111,Asia


# Build a model to predict happiness rankings

In [5]:
# Set up training and test data
from sklearn.model_selection import train_test_split

y=data['Happiness_level']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_train.columns.tolist())

(117, 8)
(117,)
['GDP per capita', 'Social support', 'Healthy life expectancy', 'Freedom to make life choices', 'Generosity', 'Perceptions of corruption', 'Terrorist_attacks', 'region']


## Preprocess data using Column Transformer and save fit preprocessor to ".pkl" file

In [0]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# We create the preprocessing pipelines for both numeric and categorical data.

numeric_features=X.columns.tolist()
numeric_features.remove('region')

X.loc[:,numeric_features]

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['region']

#Replacing missing values with Modal value and then one hot encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# final preprocessor object set up with ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


#Fit your preprocessor object
prediction_input_preprocessor=preprocessor.fit(X_train) 

import pickle
pickle.dump(prediction_input_preprocessor, open( "preprocessor.pkl", "wb" ) )

In [7]:
# Check shape for keras input:
prediction_input_preprocessor.transform(X_train).shape # pretty small dataset

(117, 12)

In [8]:
# Check shape for keras output:
pd.get_dummies(y_train).shape

(117, 5)

## Fit a neural network with Keras with new variables, regularization, or batch normalization

Recall that our in class example of a Keras model tended to score test data at around 50% accuracy

### a) Adding variable measuring count of terrorist attacks

In [20]:
# Original model with potentially meaningful new feature added PLUS training with gradient descent on fully samples (not mini batches)
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout,BatchNormalization
import keras
from keras.optimizers import SGD
model = Sequential()
model.add(Dense(64, input_dim=12, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation='softmax')) 
                                            
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Fitting the NN to the Training set
model.fit(prediction_input_preprocessor.transform(X_train), pd.get_dummies(y_train), 
               epochs = 500)  


Epoch 1/500
117/117 [==============================] - 0s 3ms/step - loss: 1.5827 - acc: 0.1966
Epoch 2/500
117/117 [==============================] - 0s 92us/step - loss: 1.5739 - acc: 0.2051
Epoch 3/500
117/117 [==============================] - 0s 125us/step - loss: 1.5661 - acc: 0.2479
Epoch 4/500
117/117 [==============================] - 0s 77us/step - loss: 1.5580 - acc: 0.2906
Epoch 5/500
117/117 [==============================] - 0s 77us/step - loss: 1.5501 - acc: 0.2991
Epoch 6/500
117/117 [==============================] - 0s 70us/step - loss: 1.5425 - acc: 0.3077
Epoch 7/500
117/117 [==============================] - 0s 78us/step - loss: 1.5348 - acc: 0.3419
Epoch 8/500
117/117 [==============================] - 0s 68us/step - loss: 1.5269 - acc: 0.3504
Epoch 9/500
117/117 [==============================] - 0s 93us/step - loss: 1.5191 - acc: 0.3932
Epoch 10/500
117/117 [==============================] - 0s 87us/step - loss: 1.5114 - acc: 0.4103
Epoch 11/500
117/117 [=======

In [21]:
# Does substantially better than model without new terrorism variable

# using predict_classes() for multi-class data to return predicted class index.

prediction_index=model.predict_classes(prediction_input_preprocessor.transform(X_test))

#Now lets run some code to get keras to return the label rather than the index...

# get labels from one hot encoded y_train data
labels=pd.get_dummies(y_train).columns

# Iterate through all predicted indices using map method

predicted_labels=list(map(lambda x: labels[x], prediction_index))
print(predicted_labels)

['Low', 'Average', 'Low', 'High', 'High', 'Very Low', 'Very Low', 'Very High', 'Very High', 'High', 'Very Low', 'Average', 'Average', 'High', 'High', 'High', 'High', 'Low', 'Very High', 'Very High', 'Average', 'Very High', 'Low', 'High', 'High', 'High', 'Very High', 'Very Low', 'Low', 'Low', 'Low', 'High', 'Average', 'Low', 'Average', 'Very High', 'Very High', 'Low', 'Very Low']


In [22]:



# load model_eval_metrics() function into our session to calculate metrics

import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import pandas as pd
from math import sqrt

def model_eval_metrics(y_true, y_pred,classification="TRUE"):
     if classification=="TRUE":
        accuracy_eval = accuracy_score(y_true, y_pred)
        f1_score_eval = f1_score(y_true, y_pred,average="macro",zero_division=0)
        precision_eval = precision_score(y_true, y_pred,average="macro",zero_division=0)
        recall_eval = recall_score(y_true, y_pred,average="macro",zero_division=0)
        mse_eval = 0
        rmse_eval = 0
        mae_eval = 0
        r2_eval = 0
        metricdata = {'accuracy': [accuracy_eval], 'f1_score': [f1_score_eval], 'precision': [precision_eval], 'recall': [recall_eval], 'mse': [mse_eval], 'rmse': [rmse_eval], 'mae': [mae_eval], 'r2': [r2_eval]}
        finalmetricdata = pd.DataFrame.from_dict(metricdata)
     else:
        accuracy_eval = 0
        f1_score_eval = 0
        precision_eval = 0
        recall_eval = 0
        mse_eval = mean_squared_error(y_true, y_pred)
        rmse_eval = sqrt(mean_squared_error(y_true, y_pred))
        mae_eval = mean_absolute_error(y_true, y_pred)
        r2_eval = r2_score(y_true, y_pred)
        metricdata = {'accuracy': [accuracy_eval], 'f1_score': [f1_score_eval], 'precision': [precision_eval], 'recall': [recall_eval], 'mse': [mse_eval], 'rmse': [rmse_eval], 'mae': [mae_eval], 'r2': [r2_eval]}
        finalmetricdata = pd.DataFrame.from_dict(metricdata)
     return finalmetricdata

model_eval_metrics( y_test,predicted_labels,classification="TRUE")


,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0.487179,0.479076,0.48202,0.477778,0,0,0,0


# Does regularization during model training help?

Dropout regularization can help ensure that we are not overfitting the model to training data.  

In [16]:
# Model with Dropout regularization

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout,BatchNormalization
import keras
from keras.optimizers import SGD
model = Sequential()
model.add(Dense(64, input_dim=12, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(5, activation='softmax')) 
                                            
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Fitting the NN to the Training set
model.fit(prediction_input_preprocessor.transform(X_train), pd.get_dummies(y_train), 
               epochs = 1000)  



Epoch 1/1000
117/117 [==============================] - 0s 3ms/step - loss: 1.6932 - acc: 0.1880
Epoch 2/1000
117/117 [==============================] - 0s 95us/step - loss: 1.6928 - acc: 0.1880
Epoch 3/1000
117/117 [==============================] - 0s 90us/step - loss: 1.6926 - acc: 0.1966
Epoch 4/1000
117/117 [==============================] - 0s 111us/step - loss: 1.6861 - acc: 0.1795
Epoch 5/1000
117/117 [==============================] - 0s 96us/step - loss: 1.6769 - acc: 0.2479
Epoch 6/1000
117/117 [==============================] - 0s 100us/step - loss: 1.6323 - acc: 0.1795
Epoch 7/1000
117/117 [==============================] - 0s 130us/step - loss: 1.6514 - acc: 0.2393
Epoch 8/1000
117/117 [==============================] - 0s 144us/step - loss: 1.6486 - acc: 0.1880
Epoch 9/1000
117/117 [==============================] - 0s 126us/step - loss: 1.6756 - acc: 0.1709
Epoch 10/1000
117/117 [==============================] - 0s 99us/step - loss: 1.6234 - acc: 0.2650
Epoch 11/1000
1

In [17]:
# This approach seems to do a bit better than building a model without Dropout regularization.

# using predict_classes() for multi-class data to return predicted class index.

prediction_index=model.predict_classes(prediction_input_preprocessor.transform(X_test))

#Now lets run some code to get keras to return the label rather than the index...

# get labels from one hot encoded y_train data
labels=pd.get_dummies(y_train).columns

# Iterate through all predicted indices using map method

predicted_labels=list(map(lambda x: labels[x], prediction_index))
print(predicted_labels)

model_eval_metrics( y_test,predicted_labels,classification="TRUE")


['Low', 'Average', 'Very Low', 'High', 'High', 'Very Low', 'Very Low', 'Very High', 'Very High', 'High', 'Very Low', 'Average', 'Average', 'High', 'High', 'High', 'High', 'Low', 'High', 'High', 'Average', 'Very High', 'Low', 'High', 'High', 'High', 'Very High', 'Very Low', 'Low', 'Low', 'Low', 'High', 'Average', 'Low', 'Average', 'Very High', 'Very High', 'High', 'Very Low']


,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0.538462,0.514286,0.528571,0.515556,0,0,0,0


# Does regularization during model training help?

What about L1 or L2 norm regularization?  We'll use L2 regularization in the below example.

Within keras there are three options: (default is None)
* kernel_regularizer(applied to weights),
* bias_regularizer(applied to bias unit), and 
* activity_regularizer(applied to layer activation).

In [23]:
# Model with L2 regularization
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout,BatchNormalization
from keras.optimizers import SGD
from keras.regularizers import l1
from keras.regularizers import l2
from keras.regularizers import l1_l2
model = Sequential()
model.add(Dense(64, input_dim=12, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dense(5, activation='softmax')) 
                                            
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Fitting the NN to the Training set
model.fit(prediction_input_preprocessor.transform(X_train), pd.get_dummies(y_train), 
               epochs = 1000)  

Epoch 1/1000
117/117 [==============================] - 0s 4ms/step - loss: 3.1369 - acc: 0.1538
Epoch 2/1000
117/117 [==============================] - 0s 104us/step - loss: 3.1144 - acc: 0.1966
Epoch 3/1000
117/117 [==============================] - 0s 96us/step - loss: 3.0934 - acc: 0.2222
Epoch 4/1000
117/117 [==============================] - 0s 113us/step - loss: 3.0736 - acc: 0.2222
Epoch 5/1000
117/117 [==============================] - 0s 117us/step - loss: 3.0542 - acc: 0.2479
Epoch 6/1000
117/117 [==============================] - 0s 99us/step - loss: 3.0373 - acc: 0.2479
Epoch 7/1000
117/117 [==============================] - 0s 90us/step - loss: 3.0202 - acc: 0.2650
Epoch 8/1000
117/117 [==============================] - 0s 90us/step - loss: 3.0048 - acc: 0.2821
Epoch 9/1000
117/117 [==============================] - 0s 100us/step - loss: 2.9895 - acc: 0.2906
Epoch 10/1000
117/117 [==============================] - 0s 94us/step - loss: 2.9747 - acc: 0.3162
Epoch 11/1000
11

In [24]:
# L2 Normalization helps!  

# using predict_classes() for multi-class data to return predicted class index.

prediction_index=model.predict_classes(prediction_input_preprocessor.transform(X_test))

#Now lets run some code to get keras to return the label rather than the index...

# get labels from one hot encoded y_train data
labels=pd.get_dummies(y_train).columns

# Iterate through all predicted indices using map method

predicted_labels=list(map(lambda x: labels[x], prediction_index))
print(predicted_labels)

modelevalobject=model_eval_metrics( y_test,predicted_labels,classification="TRUE")

modelevalobject

['Low', 'Average', 'Low', 'High', 'High', 'Average', 'Average', 'Very High', 'Very High', 'Very High', 'Very Low', 'Average', 'Average', 'High', 'High', 'High', 'Average', 'Low', 'High', 'Very High', 'Average', 'Very High', 'Low', 'Very High', 'High', 'Average', 'Very High', 'Very Low', 'Low', 'Low', 'Low', 'High', 'Average', 'Low', 'Average', 'Very High', 'Very High', 'Low', 'Very Low']


,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0.512821,0.506536,0.535556,0.516111,0,0,0,0


## Lastly, lets use Batch Normalization to normalize the weights 

# What happens if we standardize the values of our data with z scores (slightly adjust such that the results rarely equal zero) using Batch Normalization?

Batch normalization can be used to standardize the values of weights + bias (the resulting scalar values) before inserting the result into an activation transformation.

They can also be used to standardize the output of an activation function AFTER it has been transformed by the activation function (e.g.-sigmoid(sum of weights plus bias)

In [0]:
To add Batch normalization in keras use BatchNormalization()

In [0]:
# Model with Dropout regularization
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout,BatchNormalization
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(64, input_dim=12))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(5, activation='softmax')) 
                                            
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Fitting the NN to the Training set
model.fit(prediction_input_preprocessor.transform(X_train), pd.get_dummies(y_train), batch_size=50,
               epochs = 1000)  

Epoch 1/1000
117/117 [==============================] - 3s 27ms/step - loss: 1.9263 - acc: 0.1795
Epoch 2/1000
117/117 [==============================] - 0s 145us/step - loss: 1.7869 - acc: 0.1966
Epoch 3/1000
117/117 [==============================] - 0s 141us/step - loss: 1.5988 - acc: 0.2479
Epoch 4/1000
117/117 [==============================] - 0s 170us/step - loss: 1.5186 - acc: 0.2821
Epoch 5/1000
117/117 [==============================] - 0s 150us/step - loss: 1.4071 - acc: 0.3333
Epoch 6/1000
117/117 [==============================] - 0s 127us/step - loss: 1.3363 - acc: 0.3590
Epoch 7/1000
117/117 [==============================] - 0s 114us/step - loss: 1.3086 - acc: 0.4188
Epoch 8/1000
117/117 [==============================] - 0s 105us/step - loss: 1.2145 - acc: 0.4786
Epoch 9/1000
117/117 [==============================] - 0s 121us/step - loss: 1.1363 - acc: 0.5556
Epoch 10/1000
117/117 [==============================] - 0s 97us/step - loss: 1.1148 - acc: 0.5726
Epoch 11/10

In [0]:
# Does Batch Normalization help?  Not really (though it can help a lot in other examples)

# using predict_classes() for multi-class data to return predicted class index.

prediction_index=model.predict_classes(prediction_input_preprocessor.transform(X_test))

#Now lets run some code to get keras to return the label rather than the index...

# get labels from one hot encoded y_train data
labels=pd.get_dummies(y_train).columns

# Iterate through all predicted indices using map method

predicted_labels=list(map(lambda x: labels[x], prediction_index))

# add metrics to submittable object
modelevalobject=model_eval_metrics( y_test,predicted_labels,classification="TRUE")

modelevalobject


,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0.538462,0.485462,0.513462,0.508333,0,0,0,0


# Experimenting with depth of model (number of hidden layers)

In [0]:
# Model with L2 regularization
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout,BatchNormalization
from keras.optimizers import SGD
from keras.regularizers import l1
from keras.regularizers import l2
from keras.regularizers import l1_l2
model = Sequential()
model.add(Dense(64, input_dim=12, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dense(5, activation='softmax')) 
                                            
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Fitting the NN to the Training set
model.fit(prediction_input_preprocessor.transform(X_train), pd.get_dummies(y_train), batch_size=50,
               epochs = 1000)  

Epoch 1/1000
117/117 [==============================] - 0s 3ms/step - loss: 5.0356 - acc: 0.1966
Epoch 2/1000
117/117 [==============================] - 0s 102us/step - loss: 5.0294 - acc: 0.1966
Epoch 3/1000
117/117 [==============================] - 0s 122us/step - loss: 5.0230 - acc: 0.1966
Epoch 4/1000
117/117 [==============================] - 0s 110us/step - loss: 5.0170 - acc: 0.1966
Epoch 5/1000
117/117 [==============================] - 0s 121us/step - loss: 5.0113 - acc: 0.1966
Epoch 6/1000
117/117 [==============================] - 0s 102us/step - loss: 5.0053 - acc: 0.1966
Epoch 7/1000
117/117 [==============================] - 0s 88us/step - loss: 4.9997 - acc: 0.1966
Epoch 8/1000
117/117 [==============================] - 0s 80us/step - loss: 4.9940 - acc: 0.1966
Epoch 9/1000
117/117 [==============================] - 0s 99us/step - loss: 4.9886 - acc: 0.1966
Epoch 10/1000
117/117 [==============================] - 0s 138us/step - loss: 4.9831 - acc: 0.2051
Epoch 11/1000


In [0]:
# How does a deeper architecture affect model performance?  Not much diff't from best model using l2 regularization above.

# using predict_classes() for multi-class data to return predicted class index.

prediction_index=model.predict_classes(prediction_input_preprocessor.transform(X_test))

#Now lets run some code to get keras to return the label rather than the index...

# get labels from one hot encoded y_train data
labels=pd.get_dummies(y_train).columns

# Iterate through all predicted indices using map method

predicted_labels=list(map(lambda x: labels[x], prediction_index))


# load model_eval_metrics() function into our session to calculate metrics

import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import pandas as pd
from math import sqrt

def model_eval_metrics(y_true, y_pred,classification="TRUE"):
     if classification=="TRUE":
        accuracy_eval = accuracy_score(y_true, y_pred)
        f1_score_eval = f1_score(y_true, y_pred,average="macro",zero_division=0)
        precision_eval = precision_score(y_true, y_pred,average="macro",zero_division=0)
        recall_eval = recall_score(y_true, y_pred,average="macro",zero_division=0)
        mse_eval = 0
        rmse_eval = 0
        mae_eval = 0
        r2_eval = 0
        metricdata = {'accuracy': [accuracy_eval], 'f1_score': [f1_score_eval], 'precision': [precision_eval], 'recall': [recall_eval], 'mse': [mse_eval], 'rmse': [rmse_eval], 'mae': [mae_eval], 'r2': [r2_eval]}
        finalmetricdata = pd.DataFrame.from_dict(metricdata)
     else:
        accuracy_eval = 0
        f1_score_eval = 0
        precision_eval = 0
        recall_eval = 0
        mse_eval = mean_squared_error(y_true, y_pred)
        rmse_eval = sqrt(mean_squared_error(y_true, y_pred))
        mae_eval = mean_absolute_error(y_true, y_pred)
        r2_eval = r2_score(y_true, y_pred)
        metricdata = {'accuracy': [accuracy_eval], 'f1_score': [f1_score_eval], 'precision': [precision_eval], 'recall': [recall_eval], 'mse': [mse_eval], 'rmse': [rmse_eval], 'mae': [mae_eval], 'r2': [r2_eval]}
        finalmetricdata = pd.DataFrame.from_dict(metricdata)
     return finalmetricdata

model_eval_metrics( y_test,predicted_labels,classification="TRUE")


# add metrics to submittable object
modelevalobject=model_eval_metrics( y_test,predicted_labels,classification="TRUE")

modelevalobject

,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0.692308,0.678549,0.682778,0.691667,0,0,0,0


In [0]:
# Model with larger value for L2 regularization (allows for larger parameters) (see APM page 144 for more details)
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout,BatchNormalization
from keras.optimizers import SGD
from keras.regularizers import l1
from keras.regularizers import l2
from keras.regularizers import l1_l2
model = Sequential()
model.add(Dense(64, input_dim=12, activation='relu', kernel_regularizer=l2(0.1), bias_regularizer=l2(0.1)))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.1), bias_regularizer=l2(0.1)))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.1), bias_regularizer=l2(0.1)))

model.add(Dense(5, activation='softmax')) 
                                            
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Fitting the NN to the Training set
model.fit(prediction_input_preprocessor.transform(X_train), pd.get_dummies(y_train), batch_size=50,
               epochs = 1000)  

In [0]:
# How does a deeper architecture affect model performance?  Not much diff't from best model using l2 regularization above.

# using predict_classes() for multi-class data to return predicted class index.

prediction_index=model.predict_classes(prediction_input_preprocessor.transform(X_test))

#Now lets run some code to get keras to return the label rather than the index...

# get labels from one hot encoded y_train data
labels=pd.get_dummies(y_train).columns

# Iterate through all predicted indices using map method

predicted_labels=list(map(lambda x: labels[x], prediction_index))


# load model_eval_metrics() function into our session to calculate metrics

import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import pandas as pd
from math import sqrt

def model_eval_metrics(y_true, y_pred,classification="TRUE"):
     if classification=="TRUE":
        accuracy_eval = accuracy_score(y_true, y_pred)
        f1_score_eval = f1_score(y_true, y_pred,average="macro",zero_division=0)
        precision_eval = precision_score(y_true, y_pred,average="macro",zero_division=0)
        recall_eval = recall_score(y_true, y_pred,average="macro",zero_division=0)
        mse_eval = 0
        rmse_eval = 0
        mae_eval = 0
        r2_eval = 0
        metricdata = {'accuracy': [accuracy_eval], 'f1_score': [f1_score_eval], 'precision': [precision_eval], 'recall': [recall_eval], 'mse': [mse_eval], 'rmse': [rmse_eval], 'mae': [mae_eval], 'r2': [r2_eval]}
        finalmetricdata = pd.DataFrame.from_dict(metricdata)
     else:
        accuracy_eval = 0
        f1_score_eval = 0
        precision_eval = 0
        recall_eval = 0
        mse_eval = mean_squared_error(y_true, y_pred)
        rmse_eval = sqrt(mean_squared_error(y_true, y_pred))
        mae_eval = mean_absolute_error(y_true, y_pred)
        r2_eval = r2_score(y_true, y_pred)
        metricdata = {'accuracy': [accuracy_eval], 'f1_score': [f1_score_eval], 'precision': [precision_eval], 'recall': [recall_eval], 'mse': [mse_eval], 'rmse': [rmse_eval], 'mae': [mae_eval], 'r2': [r2_eval]}
        finalmetricdata = pd.DataFrame.from_dict(metricdata)
     return finalmetricdata

model_eval_metrics( y_test,predicted_labels,classification="TRUE")


# add metrics to submittable object
modelevalobject=model_eval_metrics( y_test,predicted_labels,classification="TRUE")

modelevalobject

,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0.589744,0.554599,0.603535,0.594444,0,0,0,0


In [0]:
# Model with best L2 regularization run for three times the epochs
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout,BatchNormalization
from keras.optimizers import SGD
from keras.regularizers import l1
from keras.regularizers import l2
from keras.regularizers import l1_l2
model = Sequential()
model.add(Dense(64, input_dim=12, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))

model.add(Dense(5, activation='softmax')) 
                                            
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Fitting the NN to the Training set
model.fit(prediction_input_preprocessor.transform(X_train), pd.get_dummies(y_train), batch_size=50,
               epochs = 3000)  

Streaming output truncated to the last 5000 lines.
117/117 [==============================] - 0s 79us/step - loss: 1.6044 - acc: 0.7436
Epoch 502/3000
117/117 [==============================] - 0s 86us/step - loss: 1.6036 - acc: 0.7436
Epoch 503/3000
117/117 [==============================] - 0s 79us/step - loss: 1.6027 - acc: 0.7607
Epoch 504/3000
117/117 [==============================] - 0s 72us/step - loss: 1.6026 - acc: 0.7436
Epoch 505/3000
117/117 [==============================] - 0s 82us/step - loss: 1.6002 - acc: 0.7350
Epoch 506/3000
117/117 [==============================] - 0s 102us/step - loss: 1.5997 - acc: 0.7350
Epoch 507/3000
117/117 [==============================] - 0s 91us/step - loss: 1.5987 - acc: 0.7350
Epoch 508/3000
117/117 [==============================] - 0s 85us/step - loss: 1.5970 - acc: 0.7265
Epoch 509/3000
117/117 [==============================] - 0s 94us/step - loss: 1.5949 - acc: 0.7436
Epoch 510/3000
117/117 [==============================] - 0s 11

In [0]:
# How does training the model for a longer period of time/epochs affect model performance?  Gets worse! We are overfitting!  Hence the importance of validation data.

# using predict_classes() for multi-class data to return predicted class index.

prediction_index=model.predict_classes(prediction_input_preprocessor.transform(X_test))

#Now lets run some code to get keras to return the label rather than the index...

# get labels from one hot encoded y_train data
labels=pd.get_dummies(y_train).columns

# Iterate through all predicted indices using map method

predicted_labels=list(map(lambda x: labels[x], prediction_index))


# load model_eval_metrics() function into our session to calculate metrics

import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import pandas as pd
from math import sqrt

def model_eval_metrics(y_true, y_pred,classification="TRUE"):
     if classification=="TRUE":
        accuracy_eval = accuracy_score(y_true, y_pred)
        f1_score_eval = f1_score(y_true, y_pred,average="macro",zero_division=0)
        precision_eval = precision_score(y_true, y_pred,average="macro",zero_division=0)
        recall_eval = recall_score(y_true, y_pred,average="macro",zero_division=0)
        mse_eval = 0
        rmse_eval = 0
        mae_eval = 0
        r2_eval = 0
        metricdata = {'accuracy': [accuracy_eval], 'f1_score': [f1_score_eval], 'precision': [precision_eval], 'recall': [recall_eval], 'mse': [mse_eval], 'rmse': [rmse_eval], 'mae': [mae_eval], 'r2': [r2_eval]}
        finalmetricdata = pd.DataFrame.from_dict(metricdata)
     else:
        accuracy_eval = 0
        f1_score_eval = 0
        precision_eval = 0
        recall_eval = 0
        mse_eval = mean_squared_error(y_true, y_pred)
        rmse_eval = sqrt(mean_squared_error(y_true, y_pred))
        mae_eval = mean_absolute_error(y_true, y_pred)
        r2_eval = r2_score(y_true, y_pred)
        metricdata = {'accuracy': [accuracy_eval], 'f1_score': [f1_score_eval], 'precision': [precision_eval], 'recall': [recall_eval], 'mse': [mse_eval], 'rmse': [rmse_eval], 'mae': [mae_eval], 'r2': [r2_eval]}
        finalmetricdata = pd.DataFrame.from_dict(metricdata)
     return finalmetricdata

model_eval_metrics( y_test,predicted_labels,classification="TRUE")


# add metrics to submittable object
modelevalobject=model_eval_metrics( y_test,predicted_labels,classification="TRUE")

modelevalobject

,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0.666667,0.65733,0.666667,0.669444,0,0,0,0


In [47]:
# Use model checkpoints with test data validation to save out best model evaluated on test data throughout fitting process:
# Here is a good source for further reading: https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/

# Model with best L2 regularization run for three times the epochs
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout,BatchNormalization
from keras.optimizers import SGD
from keras.regularizers import l1
from keras.regularizers import l2
from keras.regularizers import l1_l2
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

model = Sequential()
model.add(Dense(64, input_dim=12, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))

model.add(Dense(5, activation='softmax')) 
                                            
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# save best model given maximum val_accuracy, stop early if loss does not improve after 200 further iterations beyond best loss
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='max', verbose=1, save_best_only=True)

# Fitting the NN to the Training set
model.fit(prediction_input_preprocessor.transform(X_train), pd.get_dummies(y_train), batch_size=50,
          validation_data=(prediction_input_preprocessor.transform(X_test), pd.get_dummies(y_test)), epochs=1500, verbose=1, callbacks=[es])



Train on 117 samples, validate on 39 samples
Epoch 1/1500
117/117 [==============================] - 1s 5ms/step - loss: 3.1258 - acc: 0.1880 - val_loss: 3.1341 - val_acc: 0.1282
Epoch 2/1500
117/117 [==============================] - 0s 89us/step - loss: 3.1168 - acc: 0.2051 - val_loss: 3.1268 - val_acc: 0.1795
Epoch 3/1500
117/117 [==============================] - 0s 77us/step - loss: 3.1075 - acc: 0.2650 - val_loss: 3.1208 - val_acc: 0.1795
Epoch 4/1500
117/117 [==============================] - 0s 76us/step - loss: 3.0992 - acc: 0.2906 - val_loss: 3.1135 - val_acc: 0.1795
Epoch 5/1500
117/117 [==============================] - 0s 156us/step - loss: 3.0901 - acc: 0.3077 - val_loss: 3.1069 - val_acc: 0.1795
Epoch 6/1500
117/117 [==============================] - 0s 102us/step - loss: 3.0818 - acc: 0.3162 - val_loss: 3.1007 - val_acc: 0.1795
Epoch 7/1500
117/117 [==============================] - 0s 96us/step - loss: 3.0741 - acc: 0.3504 - val_loss: 3.0952 - val_acc: 0.2051
Epoch 8/1

In [48]:
# Early stopping didn't while validating on test data did not stop the model.

#How does training the model for a longer period of time/epochs affect model performance at 1500 epochs?  Gets better!

# using predict_classes() for multi-class data to return predicted class index.

prediction_index=model.predict_classes(prediction_input_preprocessor.transform(X_test))

#Now lets run some code to get keras to return the label rather than the index...

# get labels from one hot encoded y_train data
labels=pd.get_dummies(y_train).columns

# Iterate through all predicted indices using map method

predicted_labels=list(map(lambda x: labels[x], prediction_index))


# load model_eval_metrics() function into our session to calculate metrics

import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import pandas as pd
from math import sqrt

def model_eval_metrics(y_true, y_pred,classification="TRUE"):
     if classification=="TRUE":
        accuracy_eval = accuracy_score(y_true, y_pred)
        f1_score_eval = f1_score(y_true, y_pred,average="macro",zero_division=0)
        precision_eval = precision_score(y_true, y_pred,average="macro",zero_division=0)
        recall_eval = recall_score(y_true, y_pred,average="macro",zero_division=0)
        mse_eval = 0
        rmse_eval = 0
        mae_eval = 0
        r2_eval = 0
        metricdata = {'accuracy': [accuracy_eval], 'f1_score': [f1_score_eval], 'precision': [precision_eval], 'recall': [recall_eval], 'mse': [mse_eval], 'rmse': [rmse_eval], 'mae': [mae_eval], 'r2': [r2_eval]}
        finalmetricdata = pd.DataFrame.from_dict(metricdata)
     else:
        accuracy_eval = 0
        f1_score_eval = 0
        precision_eval = 0
        recall_eval = 0
        mse_eval = mean_squared_error(y_true, y_pred)
        rmse_eval = sqrt(mean_squared_error(y_true, y_pred))
        mae_eval = mean_absolute_error(y_true, y_pred)
        r2_eval = r2_score(y_true, y_pred)
        metricdata = {'accuracy': [accuracy_eval], 'f1_score': [f1_score_eval], 'precision': [precision_eval], 'recall': [recall_eval], 'mse': [mse_eval], 'rmse': [rmse_eval], 'mae': [mae_eval], 'r2': [r2_eval]}
        finalmetricdata = pd.DataFrame.from_dict(metricdata)
     return finalmetricdata

model_eval_metrics( y_test,predicted_labels,classification="TRUE")


# add metrics to submittable object
modelevalobject=model_eval_metrics( y_test,predicted_labels,classification="TRUE")

modelevalobject

,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0.538462,0.530043,0.55873,0.531111,0,0,0,0


In [49]:
# Now we will automatically save out the best model.

# Use model checkpoints with test data validation to save out best model evaluated on test data throughout fitting process:
# Here is a good source for further reading: https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/

# Model with best L2 regularization run for three times the epochs
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout,BatchNormalization
from keras.optimizers import SGD
from keras.regularizers import l1
from keras.regularizers import l2
from keras.regularizers import l1_l2
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

model = Sequential()
model.add(Dense(64, input_dim=12, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))

model.add(Dense(5, activation='softmax')) 
                                            
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# save best model given maximum val_accuracy, stop early if loss does not improve after 200 further iterations beyond best loss
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc',mode='max', verbose=1, save_best_only=True) # evaluating val_acc maximization

# Fitting the NN to the Training set
model.fit(prediction_input_preprocessor.transform(X_train), pd.get_dummies(y_train), batch_size=50,
          validation_data=(prediction_input_preprocessor.transform(X_test), pd.get_dummies(y_test)), epochs=1500, verbose=1, callbacks=[es,mc])



Streaming output truncated to the last 5000 lines.
117/117 [==============================] - 0s 66us/step - loss: 1.9092 - acc: 0.7436 - val_loss: 2.4269 - val_acc: 0.5385

Epoch 00251: val_loss improved from 2.42762 to 2.42694, saving model to best_model.h5
Epoch 252/1500
117/117 [==============================] - 0s 107us/step - loss: 1.9074 - acc: 0.7436 - val_loss: 2.4263 - val_acc: 0.5385

Epoch 00252: val_loss improved from 2.42694 to 2.42635, saving model to best_model.h5
Epoch 253/1500
117/117 [==============================] - 0s 90us/step - loss: 1.9059 - acc: 0.7436 - val_loss: 2.4234 - val_acc: 0.5385

Epoch 00253: val_loss improved from 2.42635 to 2.42341, saving model to best_model.h5
Epoch 254/1500
117/117 [==============================] - 0s 79us/step - loss: 1.9040 - acc: 0.7436 - val_loss: 2.4278 - val_acc: 0.5128

Epoch 00254: val_loss did not improve from 2.42341
Epoch 255/1500
117/117 [==============================] - 0s 109us/step - loss: 1.9019 - acc: 0.7521 -

In [50]:
model = load_model('best_model.h5')

#Now we have automated model building such that we can choose the best model evaluated on test data 
#throughout the model building process!


# using predict_classes() for multi-class data to return predicted class index.

prediction_index=model.predict_classes(prediction_input_preprocessor.transform(X_test))

#Now lets run some code to get keras to return the label rather than the index...

# get labels from one hot encoded y_train data
labels=pd.get_dummies(y_train).columns

# Iterate through all predicted indices using map method

predicted_labels=list(map(lambda x: labels[x], prediction_index))


# load model_eval_metrics() function into our session to calculate metrics

import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import pandas as pd
from math import sqrt

def model_eval_metrics(y_true, y_pred,classification="TRUE"):
     if classification=="TRUE":
        accuracy_eval = accuracy_score(y_true, y_pred)
        f1_score_eval = f1_score(y_true, y_pred,average="macro",zero_division=0)
        precision_eval = precision_score(y_true, y_pred,average="macro",zero_division=0)
        recall_eval = recall_score(y_true, y_pred,average="macro",zero_division=0)
        mse_eval = 0
        rmse_eval = 0
        mae_eval = 0
        r2_eval = 0
        metricdata = {'accuracy': [accuracy_eval], 'f1_score': [f1_score_eval], 'precision': [precision_eval], 'recall': [recall_eval], 'mse': [mse_eval], 'rmse': [rmse_eval], 'mae': [mae_eval], 'r2': [r2_eval]}
        finalmetricdata = pd.DataFrame.from_dict(metricdata)
     else:
        accuracy_eval = 0
        f1_score_eval = 0
        precision_eval = 0
        recall_eval = 0
        mse_eval = mean_squared_error(y_true, y_pred)
        rmse_eval = sqrt(mean_squared_error(y_true, y_pred))
        mae_eval = mean_absolute_error(y_true, y_pred)
        r2_eval = r2_score(y_true, y_pred)
        metricdata = {'accuracy': [accuracy_eval], 'f1_score': [f1_score_eval], 'precision': [precision_eval], 'recall': [recall_eval], 'mse': [mse_eval], 'rmse': [rmse_eval], 'mae': [mae_eval], 'r2': [r2_eval]}
        finalmetricdata = pd.DataFrame.from_dict(metricdata)
     return finalmetricdata

model_eval_metrics( y_test,predicted_labels,classification="TRUE")


# add metrics to submittable object
modelevalobject=model_eval_metrics( y_test,predicted_labels,classification="TRUE")

modelevalobject

,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0.487179,0.477424,0.488889,0.486667,0,0,0,0


# Save keras model to onnx file.  We will use this file to make predictions within a production ready scalable REST API.

In [0]:
# Load libraries for onnx model conversion (keras to onnx)
! pip3 install keras2onnx
! pip3 install onnxruntime

     |████████████████████████████████| 225kB 2.9MB/s 
     |████████████████████████████████| 4.8MB 9.0MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 81kB 9.0MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103528 sha256=9bcbdca4d8925d8f8c10421240ed238057f644ebdc3e5be322e0ac9d417761b3
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
Successfully built fire
     |████████████████████████████████| 3.6MB 3.0MB/s 


In [0]:
#Convert keras model object to onnx and then save it to .onnx file
import os

if not os.path.exists('mymodel.onnx'):
    from keras2onnx import convert_keras
    onx = convert_keras(model, 'mymodel.onnx')
    with open("mymodel.onnx", "wb") as f:
        f.write(onx.SerializeToString())

The maximum opset needed by this model is only 9.


## Aside: Example of code similar to what is run behind the scenes within our REST api:

In [0]:
# In onnx you can make predictions in the following manner.  This is what happens behinds the scenes in our live web-application.
# the json input data is sent to a REST Api, transformed to a pandas dataframe, preprocessed, then predictions are generated from our onnx model.

import onnxruntime as rt
sess= rt.InferenceSession("mymodel.onnx")
input_name = sess.get_inputs()[0].name
bodydict={'Country or region': 'United States', 'GDP per capita': [1], 'Social support': [1], 'Healthy life expectancy': [1], 'Freedom to make life choices': [1], 'Generosity': [1], 'Perceptions of corruption': [1]}
bodynew = pd.DataFrame.from_dict(bodydict)

input_data=preprocessor.transform(bodynew).astype("float32").toarray()
input_data

array([[ 0.21686903, -0.6897203 ,  1.0740974 ,  4.0057316 ,  8.259227  ,
         8.666142  ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0. 

In [0]:
# Here is the resulting predicted probability for each of the five cats of our target variable
res = sess.run(None,  {input_name: input_data})
res[0]

array([[2.11267825e-03, 1.46210585e-02, 1.03021019e-04, 9.81635809e-01,
        1.52749766e-03]], dtype=float32)

# Submit model to live REST API @ [World Happiness Prediction Model Detail Page](http://mlsitetest.com.s3-website-us-east-1.amazonaws.com/detail/World%20Happiness%20Prediction%20Model/c6f76d3649fe11ea9b520242ac1c0002)

In [0]:
#install aimodelshare library
! pip3 install https://github.com/mikedparrott/aimodelshare/blob/master/aimodelshare-0.0.2.tar.gz?raw=true

  Created wheel for aimodelshare: filename=aimodelshare-0.0.2-cp36-none-any.whl size=5375 sha256=3fc8ed83dc8145ff1c4334a8ddd87970086acbeee96443b83b4462eca477d6e6
  Stored in directory: /root/.cache/pip/wheels/31/8d/ac/09cb6ef7374ec79e02843c347195e5478144006b11def6799a
Successfully built aimodelshare


### To submit a model you need to sign up for username and password at:
[AI Model Share Initiative Site](http://mlsitetest.com.s3-website-us-east-1.amazonaws.com/login)

# Set up necessary arguments for model submission using aimodelshare python library.

## Required information for tabular models:
* api_url ( the api url for whatever aimodelshare project you are submitting a model to)
* aws key  and password (provided for you)
* model file path
* preprocessor file path
* training data (a pandas data frame such as X_train)
* model evaluation object (we created this using the model eval metrics function above)



In [0]:
import pickle

# Loading AWS keys necessary to submit model.  Loading to object, so we don't print them out in our notebook

aws_key_password_region = pickle.load( open( "worldhappiness_modelsubmission_keys.pkl", "rb" ) ) 


In [0]:
# Example Model Pre-launched into Model Share Site
apiurl="https://btuvanmi55.execute-api.us-east-1.amazonaws.com/prod/m"
username = "your aimodelshare username here"
password = "your aimodelshare password here"

region='us-east-1'
model_filepath="mymodel.onnx"   
preprocessor_filepath="preprocessor.pkl"
preprocessor="TRUE"

trainingdata=X_train

# Set aws keys for this project (these keys give you access to collaborate on a single project)

#Importing from object that stores keys so we do not print out keys for others to see.

aws_key_password_region = pickle.load( open( "worldhappiness_modelsubmission_keys.pkl", "rb" ) )

aws_key=aws_key_password_region[0]
aws_password=aws_key_password_region[1]
region=aws_key_password_region[2]

In [0]:
# Submit your model using submit_model() function
# Works with models and preprocessors. 
import aimodelshare as ai

ai.submit_model(model_filepath=model_filepath, model_eval_metrics=modelevalobject,apiurl=apiurl, username=username, password=password, aws_key=aws_key,aws_password=aws_password, region=region, trainingdata=trainingdata,preprocessor_filepath=preprocessor_filepath,preprocessor=preprocessor)

# Now you can check the leaderboard!

In [0]:
# arguments required to get leaderboard below
apiurl="https://btuvanmi55.execute-api.us-east-1.amazonaws.com/prod/m"
username = "your aimodelshare username here"
password = "your aimodelshare password here"



In [0]:
import aimodelshare as ai

leaderboard = ai.get_leaderboard(apiurl, username, password, aws_key, aws_password, region)

LEADERBOARD RANKINGS:


,accuracy,f1_score,precision,recall,mse,rmse,mae,r2,username,model_version,avg_ranking_classification,avg_ranking_regression
10,0.692308,0.693333,0.700397,0.702778,0,0,0,0,3scman,62,2.000000,1.0
29,0.666667,0.675975,0.754286,0.700952,0,0,0,0,dhoward97,69,1.666667,1.0
63,0.641026,0.646886,0.738333,0.680952,0,0,0,0,dhoward97,68,2.666667,1.0
11,0.589744,0.594805,0.714286,0.640952,0,0,0,0,dhoward97,67,3.666667,1.0
13,0.512821,0.508060,0.625909,0.544444,0,0,0,0,Yihui_Wang,19,5.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
19,0.384615,0.343861,0.348889,0.422222,0,0,0,0,zivzach,50,19.000000,1.0
25,0.384615,0.303896,0.340260,0.425000,0,0,0,0,abhay_07,18,20.000000,1.0
12,0.333333,0.337698,0.385556,0.322222,0,0,0,0,username2,3,19.333333,1.0
7,0.333333,0.337698,0.385556,0.322222,0,0,0,0,username2,2,19.333333,1.0


In [0]:
# Build, save, and submit a sklearn model

from numpy import loadtxt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier
#Create a Gaussian Classifier
model=RandomForestClassifier(n_estimators=1000, random_state = 0)
#Train the model using the training sets y_pred=clf.predict(X_test)
model.fit(prediction_input_preprocessor.transform(X_train), y_train)
y_pred=model.predict(prediction_input_preprocessor.transform(X_test))

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
#print("Accuracy on Test Data:",metrics.accuracy_score(y_test, y_pred))

print("Random Forest Classifier's cross validation accuracy:", np.mean(cross_val_score(model, prediction_input_preprocessor.transform(X_train), y_train, cv=10)))
print("Random Forest Classifier's Test-Data prediction accuracy: {:.5f}".format(model.score(prediction_input_preprocessor.transform(X_test), y_test)))


Random Forest Classifier's cross validation accuracy: 0.6333333333333334
Random Forest Classifier's Test-Data prediction accuracy: 0.41026


In [0]:
# Save sklearn modle to pkl file
import pickle
pickle.dump(model, open( "rff_model.pkl", "wb" ) )

In [0]:
# Simply update model evaluation metric object, then change the filepaths for your preprocessor file(if new) and sklearn model to submit a sklearn model
# add metrics to submittable object
modelevalobject=model_eval_metrics(y_test,y_pred,classification="TRUE")
modelevalobject

In [0]:
# Example Model Pre-launched into Model Share Site
apiurl="https://btuvanmi55.execute-api.us-east-1.amazonaws.com/prod/m"
username = "username1"
password = "Helloworld123!"






# Set aws keys for this project (these keys give you access to collaborate on a single project)

#Importing from object that stores keys so we do not print out keys for others to see.
import pickle
aws_key_password_region = pickle.load( open( "worldhappiness_modelsubmission_keys.pkl", "rb" ) )

aws_key=aws_key_password_region[0]
aws_password=aws_key_password_region[1]
region=aws_key_password_region[2]

In [0]:
# Submit new model
import aimodelshare as ai

ai.submit_model(model_filepath=model_filepath, model_eval_metrics=modelevalobject,apiurl=apiurl, username=username, password=password, aws_key=aws_key,aws_password=aws_password, region=region, trainingdata=trainingdata,preprocessor_filepath=preprocessor_filepath,preprocessor=preprocessor)

"rff_model.pkl" has been loaded to version 12 of your prediction API.
This version of the model will be used by your prediction api for all future predictions automatically.
If you wish to use an older version of the model, please reference the getting started guide at aimodelshare.com.


In [0]:
# Check leaderboard
import aimodelshare as ai

leaderboard = ai.get_leaderboard(apiurl, username, password, aws_key, aws_password, region)

LEADERBOARD RANKINGS:


,accuracy,f1_score,precision,recall,mse,rmse,mae,r2,username,model_version,avg_ranking_classification,avg_ranking_regression
12,0.717949,0.713796,0.719444,0.725000,0,0,0,0,3scman,70,1.666667,1.0
31,0.666667,0.675975,0.754286,0.700952,0,0,0,0,dhoward97,69,2.333333,1.0
11,0.692308,0.693333,0.700397,0.702778,0,0,0,0,3scman,62,3.000000,1.0
65,0.641026,0.646886,0.738333,0.680952,0,0,0,0,dhoward97,68,3.333333,1.0
13,0.589744,0.594805,0.714286,0.640952,0,0,0,0,dhoward97,67,4.666667,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
21,0.384615,0.343861,0.348889,0.422222,0,0,0,0,zivzach,50,20.666667,1.0
27,0.384615,0.303896,0.340260,0.425000,0,0,0,0,abhay_07,18,21.666667,1.0
14,0.333333,0.337698,0.385556,0.322222,0,0,0,0,username2,3,21.000000,1.0
8,0.333333,0.337698,0.385556,0.322222,0,0,0,0,username2,2,21.000000,1.0


In [0]:
! # Live REST API example for tabular data!
! curl -X POST -H "Content-Type: application/json" -d '{"data":{"Region": "test", "GDP per capita": [10000],"Social support": [1],"Healthy life expectancy": [1],"Freedom to make life choices": [1],"Generosity": [1], "Perceptions of corruption": [-1000]}}' "https://btuvanmi55.execute-api.us-east-1.amazonaws.com/prod/m"

{"message": "Internal server error"}